# MetOncoFit Example
**Author:** Scott Campit

## Summary
This notebook generates the cancer tissue models discussed in Oruganty, K., Campit, S. E., Mamde, S., Lyssiotis, C. A., & Chandrasekaran, S. (2020). Common biochemical properties of metabolic genes recurrently dysregulated in tumors. Cancer & metabolism, 8(1), 1-15.